In [ ]:
pip install python-telegram-bot google-auth google-auth-oauthlib google-auth-httplib2 gspread
pip install oauth2client

In [2]:
import logging
import gspread
import json
from oauth2client.service_account import ServiceAccountCredentials
from telegram import Update
from telegram.ext import Application, MessageHandler, filters, CallbackContext

In [3]:
sheetname = 'test-1'
sheetname.strip()

'test-1'

In [11]:
import oauth2client
gspread.__version__

'6.1.4'

In [14]:
# Open and read the JSON file
with open("other_creds.json", "r") as file:
    other_creds = json.load(file)

# Telegram Bot Token
BOT_TOKEN = other_creds['access_token_telegram_bot']

In [19]:
file_name = "Bank Transfers 2025"
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/spreadsheets",
         "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]



In [20]:
# Function to get the last N transactions from a sheet
def get_last_n_transactions(worksheet, n):
    col_b_values = worksheet.col_values(2)  # Fetch all values from column B (dates)
    last_row = len(col_b_values)  # Last filled row index
    
    if last_row < 2:  # Assuming row 1 is headers
        return []

    start_row = max(2, last_row - n + 1)  # Ensure we don't go below row 2
    return worksheet.get(f"B{start_row}:D{last_row}")  # Fetch last N rows from columns B, C, and D



In [25]:
creds = ServiceAccountCredentials.from_json_keyfile_name("your_google_creds.json", scope)
client = gspread.authorize(creds)
worksheet = client.open(file_name).worksheet('icici')


In [35]:
transactions = get_last_n_transactions(worksheet,5)
transactions

[['27-Feb', '₹6,262.00', 'tetsts'],
 ['2025-02-27', '₹6,262.00', 'tetsts'],
 ['27-Feb'],
 ['2025-02-27', '₹6,262.00', 'tetsts'],
 ['27-Feb', '', 'gs']]

In [43]:
msg = f"🔹 **Last 5 Transactions in 5':**\n"
for entry in reversed(transactions):  # Reverse for latest-first order
    date = (entry[0] if len(entry) > 0 else "-").ljust(12)  # Pad to 12 chars
    amount = (entry[1] if len(entry) > 1 else "-").ljust(12)  # Pad to 12 chars
    description = entry[2] if len(entry) > 2 else "-"  # No padding for description

    msg += f" {date} | {amount} | {description}\n"


print(msg)

🔹 **Last 5 Transactions in 5':**
 27-Feb       |              | gs
 2025-02-27   | ₹6,262.00    | tetsts
 27-Feb       | -            | -
 2025-02-27   | ₹6,262.00    | tetsts
 27-Feb       | ₹6,262.00    | tetsts



In [ ]:
# Telegram Bot Token
BOT_TOKEN = "XX"

In [15]:

# Function to handle incoming messages
def log_expense(update: Update, context: CallbackContext) -> None:
    text = update.message.text
    try:
        # Expecting format: source,amount,category
        source, amount, category = text.split(",")
        amount = float(amount)  # Convert amount to number
        
        # Append data to Google Sheet
        sheet.append_row([update.message.date.strftime("%Y-%m-%d"), source.strip(), amount, category.strip()])
        
        update.message.reply_text(f"✅ Expense Logged: {source} spent {amount} on {category}")
    except Exception as e:
        update.message.reply_text("❌ Invalid format. Please use: `source,amount,category`")


In [18]:
def main():
    app = Application.builder().token(BOT_TOKEN).build()
    app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, log_expense))
    
    print("Bot is running...")
    app.run_polling()

In [19]:
main()

Bot is running...


RuntimeError: Cannot close a running event loop